In [1]:
import sqlite3
import pandas as pd
from pandas.core.frame import DataFrame
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None

from matplotlib import pyplot as plt
from time import sleep

import matplotlib


# import gnuplotlib as gp
import numpy as np

import sys

In [2]:
pd.set_option('display.max_colwidth', None)

In [3]:
import requests

In [4]:
def get_df(f_path: str = "/home/kefei/.volttron/agents/"
                       "ba3caba2-2e44-49a3-a771-eaccf6b7bc3e/sqlhistorianagent-4.0.0/data/platform.historian.sqlite"
           ) -> pd.DataFrame:
    # f_path = "/home/kefei/.volttron/agents/85736715-3354-4b70-94d7-eb2ca6e88e48/sqlhistorianagent-4.0.0/data/platform.historian.sqlite"
    con = sqlite3.connect(f_path)

    # df = pd.read_sql_query("SELECT name FROM sqlite_schema WHERE type ='table' AND name NOT LIKE 'sqlite_%'", con)
    # df.head()

    df_data = pd.read_sql_query(sql="select * from data", con=con)
    # df_data

    df_topics = pd.read_sql_query(sql="select * from topics", con=con)
    # df_topics
    con.commit()  # to make sure refresh

    df_join = pd.merge(df_data, df_topics, on='topic_id')
    # df_join.head()

    return df_join

In [35]:
df = get_df()
df.tail()

,ts,topic_id,value_string,topic_name,metadata
49763,2022-05-18T19:51:30.023592+00:00,88,2,campus/building/user-fake/SampleLong1,"{""units"": ""1"", ""type"": ""integer"", ""tz"": ""US/Pacific""}"
49764,2022-05-18T19:51:35.020461+00:00,88,1,campus/building/user-fake/SampleLong1,"{""units"": ""1"", ""type"": ""integer"", ""tz"": ""US/Pacific""}"
49765,2022-05-18T19:51:40.021101+00:00,88,7,campus/building/user-fake/SampleLong1,"{""units"": ""1"", ""type"": ""integer"", ""tz"": ""US/Pacific""}"
49766,2022-05-18T19:51:45.024189+00:00,88,3,campus/building/user-fake/SampleLong1,"{""units"": ""1"", ""type"": ""integer"", ""tz"": ""US/Pacific""}"
49767,2022-05-18T19:51:50.022036+00:00,88,2,campus/building/user-fake/SampleLong1,"{""units"": ""1"", ""type"": ""integer"", ""tz"": ""US/Pacific""}"


In [72]:
# inspect here
df = get_df()
df.sort_values('ts', ascending=False).head(8)


,ts,topic_id,value_string,topic_name,metadata
74003,2022-05-19T20:42:30.021358+00:00,88,11,campus/building/user-fake/SampleLong1,"{""units"": ""1"", ""type"": ""integer"", ""tz"": ""US/Pacific""}"
67431,2022-05-19T20:42:30.021358+00:00,86,5.66553889764798e-16,campus/building/user-fake/EKG,"{""units"": ""degree"", ""type"": ""integer"", ""tz"": ""US/Pacific""}"
64145,2022-05-19T20:42:30.021358+00:00,85,false,campus/building/user-fake/Heartbeat,"{""units"": ""1"", ""type"": ""integer"", ""tz"": ""US/Pacific""}"
70717,2022-05-19T20:42:30.021358+00:00,87,274.82545539901963,campus/building/user-fake/OutsideAirTemperature1,"{""units"": ""F"", ""type"": ""integer"", ""tz"": ""US/Pacific""}"
60859,2022-05-19T20:42:27.937023+00:00,84,false,campus/building/restapi/random_bool,"{""units"": ""1"", ""type"": ""integer"", ""tz"": ""US/Pacific""}"
54325,2022-05-19T20:42:27.937023+00:00,82,24584.8536,campus/building/restapi/bitcoin_gbp,"{""units"": ""gbp"", ""type"": ""integer"", ""tz"": ""US/Pacific""}"
51058,2022-05-19T20:42:27.937023+00:00,81,30083.372,campus/building/restapi/bitcoin_usd,"{""units"": ""usd"", ""type"": ""integer"", ""tz"": ""US/Pacific""}"
57592,2022-05-19T20:42:27.937023+00:00,83,28935.5409,campus/building/restapi/bitcoin_eur,"{""units"": ""eur"", ""type"": ""integer"", ""tz"": ""US/Pacific""}"


In [26]:
from IPython.display import display, clear_output

In [71]:
# inspect here

topic_prfix = "campus/building/restapi/"
topic_prfix = "campus/building/user-fake/"
# topic_prfix = "campus/building/"

df = get_df()
df_filetered = df.sort_values('ts', ascending=False)
df_display = df_filetered[df_filetered.topic_name.str.contains(topic_prfix)].head(8)
df_display

old_index = df_display.index
old_index
# df_display.loc[old_index,:]
df_display.loc[old_index,:]

count = 0
while count < 500:
    df = get_df()
    df_filetered = df.sort_values('ts', ascending=False)
    df_display = df_filetered[df_filetered.topic_name.str.contains(topic_prfix)].head(8)
    df_display
    new_index = df_display.index.copy()
    index_diff = new_index.difference(old_index)
    old_index = new_index
    if len(index_diff) > 0:
#         print(df_filetered.loc[index_diff].head(8))
#         print(df_display)
        display(df_display.loc[index_diff, :])
    else:
        print("no new data published. Count at: ", count)
    count += 1
#     print("count", count)
    sleep(5)
    clear_output(wait=True)

no new data published. Count at:  2


KeyboardInterrupt: 

In [55]:


topic_prfix = "campus/building/restapi/"
topic_prfix = "campus/building/user-fake/"
topic_prfix = "campus/building/"

# inspect here
df = get_df()
df_filetered = df.sort_values('ts', ascending=False)
df_filetered = df_filetered[df_filetered.topic_name.str.contains(topic_prfix)].head(8)
df_play

# old_index = df_play.index
# count = 500
# while count > 0:
#     df = get_df()
#     df_filetered = df.sort_values('ts', ascending=False)
#     df_play = df_filetered[df_filetered.topic_name.str.contains(topic_prfix)].head(8)
#     df_play
#     new_index = df_play.index.copy()
#     index_diff = new_index.difference(old_index)
#     old_index = new_index
#     if len(index_diff) > 0:
# #         print(df_filetered.loc[index_diff].head(8))
# #         print(df_display)
#         display(df_display)
#     else:
#         print("no new data published. Count at: ", count)
#     count -= 1
# #     print("count", count)
#     sleep(2)
#     clear_output(wait=True)

,ts,topic_id,value_string,topic_name,metadata
51543,2022-05-19T16:43:20.021760+00:00,88,10,campus/building/user-fake/SampleLong1,"{""units"": ""1"", ""type"": ""integer"", ""tz"": ""US/Pacific""}"
51068,2022-05-19T16:43:20.021760+00:00,87,-2.8199460132420455,campus/building/user-fake/OutsideAirTemperature1,"{""units"": ""F"", ""type"": ""integer"", ""tz"": ""US/Pacific""}"
50593,2022-05-19T16:43:20.021760+00:00,86,0.8660254037844387,campus/building/user-fake/EKG,"{""units"": ""degree"", ""type"": ""integer"", ""tz"": ""US/Pacific""}"
50118,2022-05-19T16:43:20.021760+00:00,85,true,campus/building/user-fake/Heartbeat,"{""units"": ""1"", ""type"": ""integer"", ""tz"": ""US/Pacific""}"
48717,2022-05-19T16:43:17.725933+00:00,82,24602.4337,campus/building/restapi/bitcoin_gbp,"{""units"": ""gbp"", ""type"": ""integer"", ""tz"": ""US/Pacific""}"
49643,2022-05-19T16:43:17.725933+00:00,84,false,campus/building/restapi/random_bool,"{""units"": ""1"", ""type"": ""integer"", ""tz"": ""US/Pacific""}"
49180,2022-05-19T16:43:17.725933+00:00,83,28956.2321,campus/building/restapi/bitcoin_eur,"{""units"": ""eur"", ""type"": ""integer"", ""tz"": ""US/Pacific""}"
48254,2022-05-19T16:43:17.725933+00:00,81,30104.884,campus/building/restapi/bitcoin_usd,"{""units"": ""usd"", ""type"": ""integer"", ""tz"": ""US/Pacific""}"


In [11]:
df_hist = df_filetered[df_filetered.topic_name.str.contains(topic_prfix)].head(8)
df_hist_2 = df_filetered[df_filetered.topic_name.str.contains(topic_prfix)].head(7)

In [12]:
df_hist_2.index

Int64Index([48243, 48183, 48303, 48363, 48362, 48242, 48182], dtype='int64')

In [14]:
df_hist.index.difference(df_hist_2.index) # order matters

Int64Index([48302], dtype='int64')

In [15]:
df_hist_2.index.difference(df_hist.index)

Int64Index([], dtype='int64')